# Bayesian Logistic Regression with PyMC3

In this notebook we will be using pymc3 to examine posterior probability distributions for the parameters in logistic regression for classification.

We start by importing our required libraries.

In [ ]:
# This is a simple example of using pymc3 for Bayesian inference of the parameter distribution.
# written by William F Basener
# University of Virginia

import pymc3 as pm
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

We define a function that will be helpful for plotting.  This function does little mathematically, but will give us very nice trace plots.

In [ ]:
def plot_traces(traces, retain=0):
    '''
    Convenience function:
    Plot traces with overlaid means and values
    '''

    ax = pm.traceplot(traces[-retain:],
                      lines=tuple([(k, {}, v['mean'])
                                   for k, v in pm.summary(traces[-retain:]).iterrows()]))

    for i, mn in enumerate(pm.summary(traces[-retain:])['mean']):
        ax[i,0].annotate('{:.2f}'.format(mn), xy=(mn,0), xycoords='data'
                    ,xytext=(5,10), textcoords='offset points', rotation=90
                    ,va='bottom', fontsize='large', color='#AA0022')

Load the data and look a the data frame.

In [ ]:
data = pd.read_csv('../input/iris-flower-dataset/IRIS.csv')
data = data[['sepal_length','sepal_width','petal_length','petal_width','species']]
print(np.unique(data['species']))
data.describe()


In [ ]:
data.loc[data['species'] == 'Iris-setosa',:].mean()*10

In [ ]:
data.loc[data['species'] == 'Iris-setosa',:].cov()*10

We are using the seaborn plottng library.  This enables a very nice pairs plot for our classes.

In [ ]:
g = sns.pairplot(data, hue="species", palette="husl", markers=["o", "s", "D"])

Now we perform our MCMC computaiton.  With pymc3, this is very easy.  We use the usual "with" declaration for pymc3, then use glm for our logistic model and just have to specidfy the formula, the data, and the family.  The family is what tells pymc3 that this will be logistic regression.

We are going to use the default priors for GLM coefficients from PyMC3, which is $p(\theta)=N(0,10^{12}I)$.  These are very weak priors.

In [ ]:
with pm.Model() as model:
    pm.glm.GLM.from_formula(formula = 'species ~ sepal_length + sepal_width + petal_length + petal_width', 
                            data = data, 
                            family = pm.glm.families.Binomial())

    trace = pm.sample(1000)

In [ ]:
## I added this out of interest - I believe we discussed in class
## Defines the same logistic regression as a member of the Generalized Linear Models (GLM) family 

pm.model_to_graphviz(model)

# See: https://goldinlocks.github.io/Bayesian-logistic-regression-with-pymc3/

Here are the variable names in the output.  

In [ ]:
trace.varnames

In [ ]:
trace_simple = trace

In [ ]:
pm.summary(trace)

Here is our nice custom traceplot using the function plot_traces we defined previously.

In [ ]:
# PyMC3 trace is similar to an object that contains the output from MCMC sampling
plot_traces(trace)

In [ ]:
# The creates a matplotlib plot, so we can modify with standard matplotlib commands
pm.plots.forestplot(trace, figsize=(12, 5))
plt.grid()  # add a grid to the plot

In [ ]:
# Seaborn joint plot which plots datapairs for easy comparison

plt.figure(figsize=(9,7))
sns.jointplot(trace['petal_length'], trace['petal_width'], kind="hex", color="#4CB391")
plt.xlabel("petal_length")
plt.ylabel("petal_width");
plt.show()

plt.figure(figsize=(9,7))
sns.jointplot(trace['sepal_length'], trace['sepal_width'], kind="hex", color="#4CB391")
plt.xlabel("sepal_length")
plt.ylabel("sepal_width");
plt.show()

plt.figure(figsize=(9,7))
sns.jointplot(trace['sepal_length'], trace['petal_width'], kind="hex", color="#4CB391")
plt.xlabel("petal_length")
plt.ylabel("sepal_width");
plt.show()

plt.figure(figsize=(9,7))
sns.jointplot(trace['sepal_length'], trace['petal_length'], kind="hex", color="#4CB391")
plt.xlabel("sepal_length")
plt.ylabel("petal_width");
plt.show()

# Q1
## The results of our logistic regression model are expressed via three plots: a PyMC3 traceplot, a matplotlib forestplot, and a seaborn joint plot. In the trace plot, the predictor variables that are most likely to be zero are those with whose marginalized distribution plot (the plots on the left-hand side in the traceplot) includes zero. In this case, the sepal length and sepal width plots both include zero. Accordingly, they both have somelikelihood that coefficient could be zero. It's possible that sepal length could be slightly more likely to be zero because zero is slightly closer (visually) towards the center of the distribution that in the sepal width plot.

## A similar conclusion can be drwn from examining the credible intervals in the forest plot. Both chains in the sepal length credible interval 'touch' the zero line. In the sepal width plot, one of the two chains touches the zero line, and the othert is very close to zero and much closer than any of the other variables' chains.

## The jointplot plots two variables together. The darker hexagons in the plot indicate some relationship between the two dimensions. The darkest areas reflect the highest probabilities. Similar to the plots above, the plots that show darker hexes around or at zero have an increased probability of being zero. This can expressly be seen in the second plot (sepal length x sepal width) where colored/darker hexes are on around around zero on both axes.

### See: https://docs.exoplanet.codes/en/latest/tutorials/intro-to-pymc3/

### Question 1 - Appendix -  I worked a little bit ahead with below.

In [ ]:
## Run Automatic Differentation Variational Inference (ADVI)
## ADVI is an alternative method to CAVI to maximize the ELBO

from pymc3.variational.callbacks import CheckParametersConvergence
with model:
    callback = CheckParametersConvergence(diff='absolute')
    approx = pm.fit(n=1000, callbacks=[callback])
    
# See: https://luiarthur.github.io/statorial/varinf/introvi/
# See: https://goldinlocks.github.io/Bayesian-logistic-regression-with-pymc3/

In [ ]:
# Persist result
from pathlib import Path
import pickle
from collections import OrderedDict
with open('model_advi.pkl', 'wb') as buff:
    pickle.dump({'model': model,
                 'approx': approx}, buff)

In [ ]:
# Draw samples from the approximated distribution to obtain a trace object as above for the MCMC sampler:
trace_advi = approx.sample(200)

In [ ]:
pm.summary(trace_advi)

In [ ]:
# Visualize the covariance structure of the model
# Professor Basener - I have some questions about this

import arviz as az

az.plot_pair(trace_advi, figsize=(8, 8));

# Part 2. Logistic Regression for Predicting Coronary Heart Disease
# Base Case - Logistic Regression

Now lets apply some Bayesian Regression techniques to a healthcare problem of determining risk for Coronary Heart Disease.  Logistic Regression is a great method for this problem because it provides esitmates of probability of heart disease and the Bayesian analysis provides insight into uncertainty and importance of the different predicotr variables.

In [ ]:
chd_data = pd.read_csv("../input/coronary-heart-disease/CHDdata.csv")
chd_data.head()

In [ ]:
chd_data.describe()

In [ ]:
# Standardize the data (mean for each numerical variable of zero, standard deviation of one.)
for key in chd_data.keys()[0:9]:
    try:
        print("Standardizing "+key+".")
        chd_data[key] = chd_data[key] - np.mean(chd_data[key])
        chd_data[key] = chd_data[key] / np.std(chd_data[key])
    except:
        print("Predictor "+key+" cannot be standardized (probably a categorical variable).")
chd_data.describe()

In [ ]:
# Lets check the mean of each class to get a first look at the separation
print("Mean for CHD Positive:")
print(np.array([chd_data[chd_data.chd == 1].mean()[0:8]]))
print("Mean for CHD Negative:")
print(np.array([chd_data[chd_data.chd == 0].mean()[0:8]]))

In [ ]:
chd_data.head()

# Create Logistic Regression model with CHD dataset - with priors

In [ ]:
with pm.Model() as model:
    # Define priors for intercept & regression coefficients
    # x5 is categorical predictor 'famhist' - using uninformative prior
    # Note however the source footnoted below - "there is no such thing as
    # an uninformative (beta (prior)"
    
    priors = {
        "Intercept": pm.Normal.dist(mu=0.0, sigma=1.0),
        "sbp": pm.Normal.dist(mu=0.0, sigma=1.0),
        "tobacco": pm.Normal.dist(mu=0.0, sigma=1.0),
        "ldl": pm.Normal.dist(mu=0.0, sigma=1.0),
        "adiposity": pm.Normal.dist(mu=0.0, sigma=1.0),
        "famhist": pm.Beta.dist(alpha=1.0, beta=1.0),
        "typea": pm.Normal.dist(mu=0.0, sigma=1.0),
        "obesity": pm.Normal.dist(mu=0.0, sigma=1.0),
        "alcohol": pm.Normal.dist(mu=0.0, sigma=1.0),
        "age": pm.Normal.dist(mu=0.0, sigma=1.0),
    }
    pm.glm.GLM.from_formula(formula = 'chd ~ sbp + tobacco + ldl + adiposity + famhist + typea + obesity + alcohol + age', 
                            data = chd_data, 
                            family = pm.glm.families.Normal())

    #trace = pm.sample(5000) 
    approx = pm.fit(50000, method = 'advi')
    
# See: https://stats.stackexchange.com/questions/297901/choosing-between-uninformative-beta-priors
# See: https://docs.pymc.io/projects/examples/en/latest/generalized_linear_models/GLM-out-of-sample-predictions.html?highlight=family
# See: https://docs.pymc.io/en/v3/pymc-examples/examples/generalized_linear_models/GLM-model-selection.html?highlight=family

In [ ]:
advi_elbo = pd.DataFrame(
    {'ELBO': -approx.hist,
     'n': np.arange(approx.hist.shape[0])})

_ = sns.lineplot(y='ELBO', x='n', data=advi_elbo)

In [ ]:
trace_VI = approx.sample(draws=5000)

In [ ]:
plot_traces(trace_VI)

In [ ]:
# The creates a matplotlib plot, so we can modify with standard matplotlib commands
pm.plots.forestplot(trace_VI, figsize=(12, 5))
plt.grid()  # add a grid to the plot

In [ ]:
pm.summary(trace_VI).round(2)

# Original Logistic Regression Model - no priors

In [ ]:
with pm.Model() as model:
    pm.glm.GLM.from_formula(formula = 'chd ~ sbp + tobacco + ldl + adiposity + typea + obesity + alcohol + age + famhist', 
                            data = chd_data, 
                            family = pm.glm.families.Binomial())

    #trace = pm.sample(5000) 
    approx = pm.fit(50000, method = 'advi')

In [ ]:
advi_elbo = pd.DataFrame(
    {'ELBO': -approx.hist,
     'n': np.arange(approx.hist.shape[0])})

_ = sns.lineplot(y='ELBO', x='n', data=advi_elbo)

In [ ]:
trace_VII = approx.sample(draws=5000)

In [ ]:
plot_traces(trace_VII)

In [ ]:
# The creates a matplotlib plot, so we can modify with standard matplotlib commands
pm.plots.forestplot(trace_VII, figsize=(12, 5))
plt.grid()  # add a grid to the plot

In [ ]:
pm.summary(trace_VII).round(2)